In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

In [36]:
from modelling.nn import NeuralNetwork
nn = NeuralNetwork()

Input to neural network seems like it should be amount of features being polynomial features, at least according to this: https://medium.com/analytics-vidhya/polynomial-regression-with-keras-ef1797b39b88
As such I'll just create the same design matrix and feed it into the network.

In [37]:
from sklearn.model_selection import  train_test_split
from data.create_dataset import *
from sklearn.preprocessing import StandardScaler
X, z = create_dataset('../data/raw/SRTM_data_Norway_1.tif',degree=4)
X_train, X_test, z_train, z_test = train_test_split(X,z, test_size=0.2)
Xscaler = StandardScaler().fit(X_train)
zscaler = StandardScaler().fit(z_train)

X_train = Xscaler.transform(X_train)
z_train = zscaler.transform(z_train)

In [38]:
nn.initalize_layers(X_train.shape)

In [39]:
for layer in nn.layers:
    print(layer.weights.shape)

(15, 50)
(50, 1)


In [51]:
nn.fit(X_train,z_train)

(100, 1)


ValueError: operands could not be broadcast together with shapes (100,100) (50,50) (100,100) 